In [1]:
#################################
# filenames
#################################

number = 49

probs_file = 'newprobs'+str(number)+'.json'
filename3s = 'newprobs'+str(number)+'genre.json'
filename3se = 'newprobs'+str(number)+'e.txt'
histo_file = 'histo'+str(number)+'.txt'
url_dict_file = 'url.json'


##############################
# setup
##############################

import requests
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import random

tstart = time.time()
print(tstart)

api_url = 'https://api.hooktheory.com/v1/'
login = {"username": "jl8429",
         "password": ""}

r = requests.post(api_url+'users/auth', data=login)
#print('r', r)
#print(r.json())
'''
{'id': 136206, 'username': 'myUsername', 'activkey': 'myKey'}
'''

activkey = '0e11e1d65a021d0a883f309f813d74ad'
header = {"Authorization": "Bearer "+activkey}


#################################
# RELOAD url_dict
#################################

with open(url_dict_file, 'r') as f:
    url_dict = json.load(f)

big_dict = {}


#################################
# continue
#################################

#with open('chordProbs3new.json', "r") as f:
with open(probs_file, "r") as f:
    data3 = json.load(f)
f2 = open(filename3se, 'w')

j = 1
totals = str(len(data3))
print(totals)

histo = [0]*5000

driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")

for d in data3:
    chord1, chord2, chord3, chord4 = d['child_path'].split(",")
    mykey = str(chord1)+','+str(chord2)+','+str(chord3)+','+str(chord4)
    print(str(j)+'/'+totals, mykey)
    song_list = []
    
    # don't overwrite what we've done already
    if not big_dict.get(mykey, False):
        try:
            myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
            
            # slow down...
            time.sleep(1)
            if str(myData)=="<Response [429]>":
                #print("too many requests")
                time.sleep(11)
                myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
                if str(myData)=="<Response [429]>":
                    time.sleep(11)
                    print("too many requests")
            
            page = 1
            keepGoing = True
            #while myData.content and myData.content != b'No songs match this chord progressionnull':
            while keepGoing:
                # can't parse if it's this byte string
                if myData.content != b'No songs match this chord progressionnull':
                    #print('page', page)
                    # parse each song
                    try:
                        my_json = myData.json()
                        songs = len(my_json)
                        #print('songs len', songs)
                        if songs:
                            # for each song, pull the genre, add it to the url_dict and big_dict, then write it to the file
                            for i in range(songs):
                                url = my_json[i]['url'].split("#")[0]
                                #print(url)
                                # check if already in url_dict to save us from having to load the page slowly and parse
                                if url in url_dict:
                                    #print(url, url_dict[url])
                                    my_json[i]['genre'] = url_dict[url]
                                # if not in dict, we need to parse page and add to dict
                                else:
                                    #time.sleep(1)
                                    driver.get(url)
                                    html = driver.page_source
                                    soup = BeautifulSoup(html, 'html.parser')
                                    try:
                                        #genre = soup.find_all(['multiselect'])[0].find('a').contents[0]
                                        # some songs don't have a genre and some have multiple genres...
                                        genres = soup.find_all(['multiselect'])[0].find_all('a')
                                        gk = 0
                                        glist = []
                                        for g in genres:
                                            # .find_all() will return a lot of garbage, but the genres will have a length of 1
                                            if len(g) > 1:
                                                # if even the first is garbage, it has no genre
                                                if gk == 0:
                                                    my_json[i]['genre'] = ["Unknown"]
                                                    url_dict[url] = ["Unknown"]
                                                break
                                            else:
                                                glist.append(g.contents[0])
                                            gk += 1
                                        my_json[i]['genre'] = glist
                                        url_dict[url] = glist
                                    except:
                                        print('3. Unable to extract genre for', url)
                                        my_json[i]['genre'] = ["Unknown"]
                                        url_dict[url] = ["Unknown"]

                                #driver.close()
                                # clean up and append song to list
                                my_json[i].pop('section', None)
                                my_json[i]['url'] = url
                                #f.write('\n'+str(my_json[i])+',')
                                #print(my_json[i])
                                song_list.append(my_json[i])

                            if songs == 20:
                                #keepGoing = True
                                page += 1
                                myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                # slow down...
                                time.sleep(1)
                                if str(myData)=="<Response [429]>":
                                    #print("too many requests")
                                    time.sleep(11)
                                    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                    if str(myData)=="<Response [429]>":
                                        time.sleep(11)
                                        print("too many requests")
                            else:
                                # update histogram of how many songs per chord progression
                                index = page*20 - 20 + songs
                                print('histo1', index)
                                histo[index] += 1
                                keepGoing = False
                        else:
                            # update histogram of how many songs per chord progression
                            index = page*20 - 20
                            print("histo2", index)
                            histo[index] += 1
                            keepGoing = False

                    except:
                        # for some reason, page=20 doesn't work on 1-5-6-4... there are other data corruptions too
                        print('2. Unable to extract for', chord1, chord2, chord3, chord4, "page", page)
                        f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
                        f2.close()
                        f2 = open(filename3se, 'a')

                        page += 1
                        myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                        # slow down...
                        time.sleep(1)
                        if str(myData)=="<Response [429]>":
                            #print("too many requests")
                            time.sleep(11)
                            myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                            if str(myData)=="<Response [429]>":
                                time.sleep(11)
                                print("too many requests")
                else:
                    print("No songs match this chord progression", mykey)
                    keepGoing = False

        except:
            print('1. Unable to extract for', chord1, chord2, chord3, chord4)
            f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
            f2.close()
            f2 = open(filename3se, 'a')

            # slow down
            time.sleep(1)
        
        big_dict[mykey] = song_list
    #f.close()
    #f = open(filename3s, 'a')
    #time.sleep(1)
    j += 1
    # restart the driver every 100 chord progressions
    if j%100==0:
        driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")
    
f2.close()


with open(filename3s, "w") as f:
    json.dump(big_dict, f)

tend = time.time()
print(tend)
#f.close()
print(tend-tstart)

print(histo[:21])
with open(histo_file, "w") as f:  
    f.write(str(histo))

with open(url_dict_file, "w") as f:
    json.dump(url_dict, f)

1615696879.7228677
561
1/561 164,16,5,16
histo1 1
2/561 3,1,6,4
histo1 7
3/561 4,3,57,1
histo1 2
4/561 542,57,542,543
histo1 1
5/561 5,67,1,3
histo1 1
6/561 1,2,164,5
histo1 1
7/561 5/5,5,b3,5/5
histo1 1
8/561 67,142,1,37
histo1 1
9/561 66,b7,76/6,66
histo1 1
10/561 5,2,164,26
histo1 1
11/561 6,5,27,57/6
histo1 1
12/561 564,1,4,564
histo1 4
13/561 5/5,3,4,16
histo1 1
14/561 4,365,6,365
histo1 1
15/561 66,4,46,4
histo1 1
16/561 164,1,57,1
histo1 1
17/561 4,1,5/5,6
histo1 5
18/561 46,164,5,7/6
histo1 1
19/561 b7,1,67,b7
histo1 1
20/561 6,36,26,36
histo1 1
21/561 57/2,2,57/5,5
histo1 1
22/561 1,56,6,67
histo1 1
23/561 67,164,265,27
histo1 1
24/561 6,56/6,5,L743
histo1 1
25/561 664,2,26,47/4
histo1 1
26/561 4,6,2,b7
histo1 1
27/561 47,365,164,L264
histo1 1
28/561 1,364,6,3
histo1 4
29/561 3,4,442,27
histo1 2
30/561 164,26,16,b3
histo1 1
31/561 5,67,27,6
histo1 1
32/561 5/2,2,b7,5
histo1 1
33/561 46,164,464/6,1
histo1 1
34/561 37,6,36,47
histo1 1
35/561 564,37,542,7/6
histo1 1
36/561 b3,b7,

histo1 1
293/561 2,6,27,37
histo1 2
294/561 b6,b7,1,5
histo1 6
295/561 6,57,2,564/6
histo1 1
296/561 5/6,6,27,5
histo1 3
297/561 642,L4,4,664
histo1 1
298/561 265,3,1,17
histo1 1
299/561 1,5,6,542/4
histo1 2
300/561 2,3,542,6
histo1 1
301/561 47,17,56/6,7/6
histo1 1
302/561 4,3,2,7/6
histo1 1
303/561 4,6,46,56
histo1 1
304/561 2,264,57,543
histo1 1
305/561 7,665,164,7/3
histo1 1
306/561 3,4,5,264
histo1 1
307/561 b3,b7,C2,C5
histo1 1
308/561 5/6,2,27,5
histo1 1
309/561 1,6,66,37
histo1 1
310/561 6,5,57,1
histo1 3
311/561 242,4,16,27
histo1 1
312/561 57/2,2,5,76/5
histo1 1
313/561 67,1,4,542
histo1 1
314/561 67,27,67,b67
histo1 2
315/561 1,3,36,3
histo1 1
316/561 164,5,6,164
histo1 1
317/561 5,27,1,67
histo1 2
318/561 664,6,5,4
histo1 3
319/561 4,464/4,D47,464/4
histo1 1
320/561 37,27,543,37
histo1 1
321/561 5/5,4,5/5,6
histo1 1
322/561 4,57,37,57/2
histo1 1
323/561 4,b4,1,6
histo1 7
324/561 56,46,5,4
histo1 3
325/561 1,37,2,3
histo1 1
326/561 6,D47,5,1
histo1 1
327/561 664,4,16,1
histo

In [ ]:
##########################
# list errors here
##########################

337/561 1,5,6,4
2. Unable to extract for 1 5 6 4 page 20
histo1 487
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'


In [4]:
##########################
# test errors here
##########################
mykey = '1,5,6,4'
page = 20
if page == 1:
    myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
else:
    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
print(myData.content)
#print(len(myData.json()))
#print(myData.json())

b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'


In [7]:
# ##########################
# # fix histo here
# ##########################
# histo[20] -= 1
# print(histo[:21])
# with open(histo_file, "w") as f:  
#     f.write(str(histo))

[0, 351, 68, 26, 12, 6, 3, 4, 5, 3, 2, 3, 1, 2, 0, 0, 0, 0, 2, 1, 2]
